In [1]:
import os
import sys
sys.path.append(os.path.join(os.getcwd(),"apperception"))

### IMPORTS
import cv2

from scenic_world import *
from world_executor import *
from video_util import *
from metadata_util import *
import json
import pandas as pd

from scenic_generate_df import *


In [2]:
### Let's define some attribute for constructing the world first
name = 'ScenicWorld' # world name
units = 'metrics'      # world units
user_data_dir = os.path.join("v1.0-mini")


In [ ]:
# generate dataframes from scenic json files
# takes 1min to run
df_sample_data, df_annotation = scenic_generate_df()

In [3]:
scenic_world = ScenicWorld(name=name, units=units)

In [4]:
### Ingest the camera to the world
scene_name = "scene-0061"
scenic_world = scenic_world.scenic_camera(scene_name)

In [5]:
### Call execute on the world to run the detection algorithm and save the real data to the database
recognized_scenic_world = scenic_world.recognize(scene_name, df_sample_data, df_annotation)
recognized_scenic_world.execute()

Worlds Table created successfully........
New world inserted successfully........
Camera Table created successfully........
New camera inserted successfully.........
Recognization done, saving to database......
